In [166]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
import re
password = getpass.getpass()


········


In [167]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)


# Creating query to get the needed information

In [168]:
query='''
select c.name as category,f.rental_duration,f.rental_rate,f.length,f.rating,r.rental_date,f.special_features from film f
join inventory i using(film_id)
join rental r using(inventory_id)
join film_category fc
ON fc.film_id=f.film_id
join category c using(category_id)
where year(rental_date)=2005;'''
data=pd.read_sql_query(query, engine)
data.head()

,category,rental_duration,rental_rate,length,rating,rental_date,special_features
0,Action,6,0.99,113,PG,2005-08-02 01:16:59,"Commentaries,Deleted Scenes,Behind the Scenes"
1,Action,6,0.99,113,PG,2005-08-18 04:26:54,"Commentaries,Deleted Scenes,Behind the Scenes"
2,Action,6,0.99,113,PG,2005-06-20 20:35:28,"Commentaries,Deleted Scenes,Behind the Scenes"
3,Action,6,0.99,113,PG,2005-07-09 05:01:58,"Commentaries,Deleted Scenes,Behind the Scenes"
4,Action,6,0.99,113,PG,2005-07-30 08:02:39,"Commentaries,Deleted Scenes,Behind the Scenes"


# Creating a boolean column to judge if the films are rented last month(May) 

In [169]:
data['rental_date']=pd.to_datetime(data['rental_date'],format='%y%m%d')

In [170]:
data['rented_last_month']=data['rental_date'].dt.month==5

In [171]:
data.head()

,category,rental_duration,rental_rate,length,rating,rental_date,special_features,rented_last_month
0,Action,6,0.99,113,PG,2005-08-02 01:16:59,"Commentaries,Deleted Scenes,Behind the Scenes",False
1,Action,6,0.99,113,PG,2005-08-18 04:26:54,"Commentaries,Deleted Scenes,Behind the Scenes",False
2,Action,6,0.99,113,PG,2005-06-20 20:35:28,"Commentaries,Deleted Scenes,Behind the Scenes",False
3,Action,6,0.99,113,PG,2005-07-09 05:01:58,"Commentaries,Deleted Scenes,Behind the Scenes",False
4,Action,6,0.99,113,PG,2005-07-30 08:02:39,"Commentaries,Deleted Scenes,Behind the Scenes",False


# Seperating 'special_features' into several subcolumns

In [172]:
sf_list = data['special_features'].unique()
sf_unique = []
for case in sf_list:
    list_of_features = re.findall("[^,]+",case)
    for feature in list_of_features:
        if feature in sf_unique:
            pass
        else:
            sf_unique.append(feature)

#This function is copied from Fernando 

In [173]:
for feature in sf_unique:
    data[feature] = data['special_features'].apply(lambda x: 1 if feature in x else 0)


In [174]:
data.head()

,category,rental_duration,rental_rate,length,rating,rental_date,special_features,rented_last_month,Commentaries,Deleted Scenes,Behind the Scenes,Trailers
0,Action,6,0.99,113,PG,2005-08-02 01:16:59,"Commentaries,Deleted Scenes,Behind the Scenes",False,1,1,1,0
1,Action,6,0.99,113,PG,2005-08-18 04:26:54,"Commentaries,Deleted Scenes,Behind the Scenes",False,1,1,1,0
2,Action,6,0.99,113,PG,2005-06-20 20:35:28,"Commentaries,Deleted Scenes,Behind the Scenes",False,1,1,1,0
3,Action,6,0.99,113,PG,2005-07-09 05:01:58,"Commentaries,Deleted Scenes,Behind the Scenes",False,1,1,1,0
4,Action,6,0.99,113,PG,2005-07-30 08:02:39,"Commentaries,Deleted Scenes,Behind the Scenes",False,1,1,1,0


# Droping the unnecessary columns

In [175]:
data=data.drop(['rental_date','special_features'],axis=1)

In [176]:
data.head()

,category,rental_duration,rental_rate,length,rating,rented_last_month,Commentaries,Deleted Scenes,Behind the Scenes,Trailers
0,Action,6,0.99,113,PG,False,1,1,1,0
1,Action,6,0.99,113,PG,False,1,1,1,0
2,Action,6,0.99,113,PG,False,1,1,1,0
3,Action,6,0.99,113,PG,False,1,1,1,0
4,Action,6,0.99,113,PG,False,1,1,1,0


### Checking NaN values

In [177]:
data.isna().sum()

category             0
rental_duration      0
rental_rate          0
length               0
rating               0
rented_last_month    0
Commentaries         0
Deleted Scenes       0
Behind the Scenes    0
Trailers             0
dtype: int64

# Spliting 'y' and 'x' 

In [178]:
y = data['rented_last_month']
X = data.drop('rented_last_month', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [179]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

### normalizing the train part

In [180]:
X_train_num = X_train.select_dtypes(include = np.number)


In [181]:
transformer = MinMaxScaler().fit(X_train_num) 
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)
X_train_norm.columns=X_train_num.columns
X_train_norm.head()

,rental_duration,rental_rate,length,Commentaries,Deleted Scenes,Behind the Scenes,Trailers
0,1.00,0.5,0.575540,0.0,0.0,1.0,1.0
1,0.75,0.5,0.669065,0.0,1.0,1.0,1.0
2,0.50,1.0,0.776978,0.0,1.0,1.0,0.0
3,0.00,0.0,0.683453,0.0,1.0,1.0,1.0
4,0.25,0.0,0.532374,0.0,0.0,1.0,0.0


In [182]:
X_train_categorical = X_train.select_dtypes(include = np.object)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2476\3895947676.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_categorical = X_train.select_dtypes(include = np.object)


In [183]:
X_train_categorical = X_train.select_dtypes(include = np.object)
X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=X_train_categorical.columns,
                             drop_first=True)
X_train_cat.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2476\2612659871.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_categorical = X_train.select_dtypes(include = np.object)


,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
11362,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
824,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8423,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
15235,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
10551,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [184]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)

# Logistic Regression

In [185]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [186]:
predictions = classification.predict(X_train_transformed)
classification.score(X_train_transformed, y_train)

0.928284340767594

### normalizing the test part

In [187]:
X_test_num = X_test.select_dtypes(include = np.number)

In [188]:
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)

In [189]:
X_test_norm.columns=X_test_num.columns

In [190]:
X_test_categorical = X_test.select_dtypes(include = np.object)
X_test_cat = pd.get_dummies(X_test_categorical, 
                            columns=X_test_categorical.columns,
                            drop_first=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2476\3812068431.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_categorical = X_test.select_dtypes(include = np.object)


In [191]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)

In [192]:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.9224708477781279

In [193]:
predictions

array([False, False, False, ..., False, False, False])

In [194]:
print(y_test.value_counts())

False    2927
True      246
Name: rented_last_month, dtype: int64


In [195]:
pd.Series(predictions).value_counts()
#something wrong with my model. I don't know how to improve it.

False    3173
dtype: int64